In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
video_file_path = './test1.mp4'

In [3]:
capture = cv2.VideoCapture(video_file_path)
ret, frame = capture.read()
cv2.imshow('frame',frame)
cv2.waitKey(0)


13

In [4]:
# function takes in raw image from frame and returns canny edge image

def canny_conversion(img):
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # converting to gray scale image for processing in opencv
  kernel_size = 5
  blurred_img = cv2.GaussianBlur(gray_img, (kernel_size,kernel_size), 0)  # blurring the image if the image is sharp
  canny = cv2.Canny(blurred_img, 50, 150)     # applying canny algorithm for edge detection
  return canny


In [5]:
# choosing the region of interest (triangular) where the lanes can be founded and ignoring other regions

def region_of_interest(image):
  height, width = image.shape
  mask = np.zeros_like(image)    # creating the mask of sam esize as image
  interested_region_triangle = np.array([[(200, height), (800, 350), (1200, height)]], np.int32)    # slicing the traingle for lanes ocurence for given input image
  cv2.fillPoly(mask, interested_region_triangle, 255)      # filling sliced triangle with white pixels and masking input image to filter out other image parts but the triangle
  masked_image = cv2.bitwise_and(image, mask)     # filtering the input image with the mask
  return masked_image

In [6]:
# detecting the lines through hough transform for a given image and returns potential lines for the lanes

def hough_transform(image):
  hough_lines = cv2.HoughLinesP(image, 2, np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap = 50)
  return hough_lines

In [7]:
# function that returns image with lines detection 

def display_lines(image, lines):
  line_image = np.zeros_like(image)
  if lines is not None:
    for line in lines:
      x1, y1, x2, y2 = line.reshape(4)
      cv2.line(line_image, (x1,y1),(x2,y2),(0,0,255), 7)
  return line_image    


In [8]:
def make_coordinates(image, line_parameters):
  slope, intercept = line_parameters
  y1 = image.shape[0]
  y2 = int(y1 * 3/5)
  x1 = int((y1 - intercept) / slope)
  x2 = int((y2-intercept) / slope)
  
  return np.array([x1, y1, x2, y2])


In [9]:
# finding the average of lines detected by hough transforms and outputs the single line. 

def avg_slope_intercept(image, lines):
  left_vals = []
  right_vals = []
  for line in lines:
    for x1,y1, x2, y2 in line:
      parameters = np.polyfit((x1,x2), (y1,y2), 1)      # fit the 1st degree polynomial and output the parameters for given coordinates
      slope = parameters[0]
      intercept = parameters[1]
      if slope > 0:
        right_vals.append((slope, intercept))
      else:
        left_vals.append((slope, intercept))
  
  left_line_avg = np.average(left_vals, axis = 0)
  right_line_avg = np.average(right_vals, axis = 0) 
  #print(left_line_avg,  'left')
  #print(right_line_avg , 'right')
  left_line = make_coordinates(image, left_line_avg)
  right_line = make_coordinates(image, right_line_avg)    

  return np.array([left_line, right_line])           


In [ ]:
cap = cv2.VideoCapture('test1.mp4')
while(cap.isOpened()):
  _, frame = cap.read()
  output = canny_conversion(frame)
  masked_output = region_of_interest(output)
  hough_lines = hough_transform(masked_output)
  averaged_lines = avg_slope_intercept(frame, hough_lines)
  line_image = display_lines(frame, averaged_lines)
  final_image = cv2.addWeighted(frame,0.8, line_image, 1, 1)
  cv2.imshow('video_output',final_image)
  if cv2.waitKey(1) == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()